In [1]:
import sys
import yaml
import torch
import logging
from pathlib import Path

# Add the path to the directory containing the omnicell package
# Assuming the omnicell package is in the parent directory of your notebook
sys.path.append('..')  # Adjust this path as needed

import yaml
import torch
import logging
from pathlib import Path
from omnicell.config.config import Config
from omnicell.data.loader import DataLoader
from omnicell.constants import PERT_KEY, GENE_EMBEDDING_KEY
# from omnicell.models.model_factory import get_model

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure paths
MODEL_CONFIG = "/orcd/data/omarabu/001/njwfish/omnicell/configs/models/sclambda_large_no_clip.yaml"
ETL_CONFIG = "/orcd/data/omarabu/001/njwfish/omnicell/configs/ETL/log_norm_llamaPMC7B.yaml"  # Change this to your desired ETL config
SPLIT_CONFIG = "/orcd/data/omarabu/001/njwfish/omnicell/configs/repogle_k562_essential_raw/random_splits/rs_accP_k562_ood_ss:ns_20_2_most_pert_0.1/split_0/split_config.yaml"
EVAL_CONFIG = None  # Set this if you want to run evaluations

# Load configuration
config = Config.from_yamls(MODEL_CONFIG, ETL_CONFIG, SPLIT_CONFIG, EVAL_CONFIG)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize data loader and load training data
loader = DataLoader(config)
adata, pert_rep_map = loader.get_training_data()

# Get dimensions and perturbation IDs
input_dim = adata.obsm['embedding'].shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pert_ids = adata.obs[PERT_KEY].unique()
gene_emb_dim = adata.varm[GENE_EMBEDDING_KEY].shape[1] if GENE_EMBEDDING_KEY in adata.varm else None

print(f"Data loaded:")
print(f"- Number of cells: {adata.shape[0]}")
print(f"- Input dimension: {input_dim}")
print(f"- Number of perturbations: {len(pert_ids)}")


2025-01-12 19:56:36,706 - INFO - Loading training data at path: /orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/K562_essential_raw_singlecell_01.h5ad


Using device: cuda


2025-01-12 19:56:47,380 - INFO - Loaded unpreprocessed data, # of data points: 310385, # of genes: 8563.
2025-01-12 19:56:47,381 - INFO - Preprocessing training data
/orcd/data/omarabu/001/njwfish/omnicell/notebooks/../omnicell/data/loader.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't hav

Data loaded:
- Number of cells: 279630
- Input dimension: 8563
- Number of perturbations: 1850


In [2]:
from train import get_model
model = get_model(config.get_model_name(), config.model_config, loader, pert_rep_map, input_dim, device, pert_ids, gene_emb_dim)

/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-12 19:57:51,653 - INFO - SCLambda model selected


In [3]:
model.train(adata)

2025-01-12 19:59:05,108 - INFO - Computing 32000-dimensional perturbation embeddings for 279630 cells...
/home/njwfish/miniconda3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/storage.py:85: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
2025-01-12 19:59:35,468 - INFO - Splitting data...
  0%|          | 0/200 [00:00<?, ?it/s]